In [67]:
import pandas as pd
import numpy as nd

In [68]:
#dataset
df = pd.read_csv('Combined_News_DJIA.csv')

In [69]:
df.shape

(1989, 27)

In [70]:
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [71]:
# separating dataset train 80% and test 20%
# time series data 
train = df[df['Date'] < '2015-01-01']
test = df[df['Date'] > '2014-12-31']

print train.shape
print test.shape

(1611, 27)
(378, 27)


In [72]:
# consolidate all Top25 headlines into one chunk of strings for each row/date
train_headlines = []
for row in range(0, len(train.index)):
    train_headlines.append(''.join(str(x)for x in train.iloc[row, 2:27]))

# vectorize first then concatenate

#train_headlines = ['something']

#for row in range(0, len(train.index)): 
    #basictrain = vect.fit_transform(train_headlines)
    #train_headlines.append(''.join(str(x)for x in train.iloc[row, 2:27]))

In [73]:
# import and instantiate CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

basic_train = vect.fit_transform(train_headlines)
print basic_train.shape

# dataset contains 45,865 different tokens

(1611, 45865)


In [74]:
# 1st Model - Logistic Regression

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logregbase = logreg.fit(basic_train, train['Label'])

In [75]:
# consolidate test dataset

test_headlines = []
for row in range(0, len(test.index)):
    test_headlines.append(''.join(str(x) for x in test.iloc[row, 2:27]))

# Vectorize first and then consolidate
#test_headlines = ['something']

#for row in range(0, len(test.index)): 
    #basictest = vect.transform(test_headlines)
    #test_headlines.append(''.join(str(x)for x in test.iloc[row, 2:27]))    

basictest = vect.transform(test_headlines)
predictions = logregbase.predict(basictest)


In [76]:
# prediction accuracy

pd.crosstab(test['Label'], predictions, rownames=['Actual'],colnames = ['Predicted'])

# Accuracy rate is apx 43%

Predicted,0,1
Actual,,
0,64,122
1,93,99


In [77]:
# A list of tokens in our feature set and coefficients 
# The top 10 positive and negative coefficients.

basicwords = vect.get_feature_names()
basiccoeffs = logregbase.coef_.tolist()[0]
coeffdf = pd.DataFrame({'Word' : basicwords, 
                        'Coefficient' : basiccoeffs})
coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
coeffdf.head(10)

# Top 10 positive coeff

,Coefficient,Word
36421,0.459878,self
7064,0.422543,canadian
22868,0.422332,korea
12779,0.417727,doctors
22675,0.398051,kills
45734,0.392864,zealand
37796,0.392149,so
42210,0.388293,tv
29808,0.386326,past
42176,0.386020,turn


In [78]:
coeffdf.tail(10)
# Top10 negative coeff

,Coefficient,Word
38244,-0.401858,speech
37829,-0.431717,society
40097,-0.437828,system
12169,-0.439434,did
10127,-0.460634,country
5032,-0.465274,begin
24327,-0.470292,low
36656,-0.479121,sex
35288,-0.536519,run
35602,-0.540694,sanctions


In [79]:
# create a new CountVectorizer with the n-gram parameter set to 2 instead of the default 
# value of 1.

advvect = CountVectorizer(ngram_range=(2,2))
advancedtrain = advvect.fit_transform(train_headlines)

print advancedtrain.shape

# 371,577 two-word combinations

(1611, 371577)


In [82]:
# 2-gram case
advmodel = LogisticRegression()
advmodel = advmodel.fit(advancedtrain, train["Label"])

In [83]:
# clean up test data - a chunk of strings
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))

advancedtest = advvect.transform(testheadlines)
advpredictions = advmodel.predict(advancedtest)

In [85]:
pd.crosstab(test["Label"], advpredictions, rownames=["Actual"], colnames=["Predicted"])

# Accuracy increased to 56% 

Predicted,0,1
Actual,,
0,69,117
1,49,143


In [86]:
advwords = advvect.get_feature_names()
advcoeffs = advmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})

advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
advcoeffdf.head(10)

# Top 10 positive coeff

,Coefficient,Words
25016,0.279436,and other
275433,0.278784,right to
288886,0.276995,set to
320043,0.255453,the first
125464,0.223548,forced to
158846,0.221361,in china
175420,0.217889,it has
361632,0.217877,will be
126933,0.213953,found in
146250,0.207799,have to


In [87]:
advcoeffdf.tail(10)
# Top 10 negative coeff

,Coefficient,Words
119549,-0.209019,fire on
49189,-0.209340,bin laden
156179,-0.211252,if he
222388,-0.214659,nuclear weapons
667,-0.219754,10 000
32005,-0.221851,around the
345236,-0.223192,up in
325492,-0.224133,there is
331498,-0.224201,to kill
319269,-0.318302,the country


In [51]:
# 2nd Model - Multinomial Naive Bayes

# import and instantiate a Multinomial Naive Bayes model second   
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [53]:
# train the model using train_headlines_spm
%time nb.fit(basic_train, train['Label'])

CPU times: user 6.87 ms, sys: 3.42 ms, total: 10.3 ms
Wall time: 9.16 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [62]:
# make predictions for train data

test_headlines = []
for row in range(0, len(test.index)):
    test_headlines.append(''.join(str(x)for x in test.iloc[row, 2:27]))

test_headlines_spm = vect.transform(test_headlines)
y_pred_class = nb.predict(test_headlines_spm)



In [60]:
pd.crosstab(test["Label"], y_pred_class, rownames=["Actual"], colnames=["Predicted"])

# Accuracy rate of 46%

Predicted,0,1
Actual,,
0,15,171
1,35,157
